In [1]:
!pip install -U ultralytics

In [2]:
!nvidia-smi

Sun Jul 16 18:24:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    51W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [1]:
from ultralytics import YOLO
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import tensorflow as tf
#Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
# tf.config.list_physical_devices('GPU')

[]

In [3]:
model = YOLO("yolov8x.pt")

In [4]:
from google.cloud import storage
import os

In [5]:
storage_client = storage.Client(project='gcds-oht33765u9-2023')

In [10]:
import shutil
shutil.rmtree('tmp')

FileNotFoundError: [Errno 2] No such file or directory: 'tmp'

In [11]:
!pwd

/home/jupyter


In [13]:
import os
from google.cloud import storage
# Initialize Google Cloud Storage client
storage_client = storage.Client()
# Specify your bucket name and model folder
bucket_name = "trap-camera-data"
model_folder_name = "model"
# Define the temporary local directory path within the Vertex environment
temp_model_dir = "/tmp/model/"
# os.makedirs(temp_model_dir)
# Get the list of blobs (files) in the model folder
bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix=model_folder_name + "/")  # Specify the model folder as the prefix
# Copy each blob (file) from the model folder to the temporary local directory
for blob in blobs:
    if not blob.name.endswith('/'):  # Check if the blob is a file and not a directory
        # Construct the destination path for the file in the temporary directory
        relative_path = os.path.relpath(blob.name, model_folder_name)  # Get the relative path within the model folder
        destination_path = os.path.join(temp_model_dir, relative_path)
        # Create subdirectories if needed
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        # Download the file to the destination path
        blob.download_to_filename(destination_path)
        print(f"Copied {blob.name} to {destination_path}")
# Now you can access the model files in the temporary local directory (e.g., /tmp/model)

Copied model/bestn.pt to /tmp/model/bestn.pt
Copied model/dataset_0/dataset.yaml to /tmp/model/dataset_0/dataset.yaml
Copied model/dataset_0/images/train/-tailed-macaque-macaca-nemestrina-in-nature-of-tropical-forest-in-1817885570.png to /tmp/model/dataset_0/images/train/-tailed-macaque-macaca-nemestrina-in-nature-of-tropical-forest-in-1817885570.png
Copied model/dataset_0/images/train/00000161-7af0-d279-ad6f-fffd5baf0000_4x3.png to /tmp/model/dataset_0/images/train/00000161-7af0-d279-ad6f-fffd5baf0000_4x3.png
Copied model/dataset_0/images/train/01468685.png to /tmp/model/dataset_0/images/train/01468685.png
Copied model/dataset_0/images/train/01508980.png to /tmp/model/dataset_0/images/train/01508980.png
Copied model/dataset_0/images/train/0729_JRPool_Cats_Tiger1.png to /tmp/model/dataset_0/images/train/0729_JRPool_Cats_Tiger1.png
Copied model/dataset_0/images/train/1.png to /tmp/model/dataset_0/images/train/1.png
Copied model/dataset_0/images/train/1004_rhino_hornbill_web.png to /tmp/

In [15]:
model.train(data=r"/tmp/model/dataset_0/dataset.yaml", epochs=200, project=r"/tmp/model/runs_0" )

Ultralytics YOLOv8.0.136 🚀 Python-3.10.11 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16161MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/tmp/model/dataset_0/dataset.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=tmp/model/runs_0, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimiz

In [17]:
model.export(format='onnx')
model.export(format='torchscript')
model.export(format='coreml')

requirements: Ultralytics requirement ['py-cpuinfo'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 3.2s, installed 1 package: ['py-cpuinfo']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics YOLOv8.0.136 🚀 Python-3.10.11 torch-2.0.1+cu117 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients

PyTorch: starting from tmp/model/runs_0/train2/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (130.4 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.23.4 which is incompatible.
google-api-core 1.34.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 17.6s, saved as tmp/model/runs_0/train2/weights/best.onnx (260.1 MB)

Export complete (26.3s)
Results saved to /home/jupyter/tmp/model/runs_0/train2/weights
Predict:         yolo predict task=detect model=tmp/model/runs_0/train2/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=tmp/model/runs_0/train2/weights/best.onnx imgsz=640 data=/tmp/model/dataset_0/dataset.yaml 
Visualize:       https://netron.app
Ultralytics YOLOv8.0.136 🚀 Python-3.10.11 torch-2.0.1+cu117 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients

PyTorch: starting from tmp/model/runs_0/train2/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (130.4 MB)

TorchScript: starting export with torch 2.0.1+cu117...
TorchScript: export success ✅ 10.7s, saved as tmp/model/runs_0/train2/weights/best.torchscript (260.5 MB)

Export complete (15.5s)
Results saved to /home/jupyter/tmp/model/runs_0/train2/weigh

'tmp/model/runs_0/train2/weights/best.mlmodel'

In [21]:
bucket_name = "trap-camera-data"
source_folder = "tmp/model/runs_0"
destination_folder = "model/runs_0"
files=[]
for root, _, filenames in os.walk(source_folder):
    for filename in filenames:
        files.append(os.path.join(root, filename))
bucket = storage_client.bucket(bucket_name)
for file_path in files:
    relative_path = os.path.relpath(file_path, source_folder)
    destination_path = os.path.join(destination_folder, relative_path)
    blob = bucket.blob(destination_path)
    blob.upload_from_filename(file_path)
    print(f"Transferred {file_path} to gs://{bucket_name}/{destination_path}")

Transferred tmp/model/runs_0/train2/events.out.tfevents.1689568674.umn-20230716-225759.3410.0 to gs://trap-camera-data/model/runs_0/train2/events.out.tfevents.1689568674.umn-20230716-225759.3410.0
Transferred tmp/model/runs_0/train2/val_batch1_pred.jpg to gs://trap-camera-data/model/runs_0/train2/val_batch1_pred.jpg
Transferred tmp/model/runs_0/train2/confusion_matrix.png to gs://trap-camera-data/model/runs_0/train2/confusion_matrix.png
Transferred tmp/model/runs_0/train2/P_curve.png to gs://trap-camera-data/model/runs_0/train2/P_curve.png
Transferred tmp/model/runs_0/train2/F1_curve.png to gs://trap-camera-data/model/runs_0/train2/F1_curve.png
Transferred tmp/model/runs_0/train2/train_batch0.jpg to gs://trap-camera-data/model/runs_0/train2/train_batch0.jpg
Transferred tmp/model/runs_0/train2/train_batch2.jpg to gs://trap-camera-data/model/runs_0/train2/train_batch2.jpg
Transferred tmp/model/runs_0/train2/train_batch7981.jpg to gs://trap-camera-data/model/runs_0/train2/train_batch7981.

In [2]:
from ultralytics import YOLO
model = YOLO('tmp/model/runs_0/train2/weights/best.pt')

In [5]:
pip install protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
result = model('Rhinoceros-Hornbill-picture-malaysia.JPG.13.jpg')
res_plotted = result[0].plot()
cv2.imshow('result',res_plotted)


image 1/1 /home/jupyter/Rhinoceros-Hornbill-picture-malaysia.JPG.13.jpg: 640x448 1 Rhinoceros Hornbill, 28.1ms
Speed: 5.3ms preprocess, 28.1ms inference, 146.1ms postprocess per image at shape (1, 3, 640, 448)


In [5]:
pip install 'protobuf<=3.20.1' --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-aiplatform 1.26.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-artifact-registry 1.8.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.


In [2]:
model = YOLO('best.pt')

In [11]:
from ultralytics import YOLO
import cv2
import os
import datetime

model = YOLO(r'C:\Users\HP\Desktop\best.pt')
source = r'G:\Trap\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4'
cap = cv2.VideoCapture(source)
path = r'C:\Users\HP\runs\detect\predict3'
frame_no = 0
video_name = source.split("\\")[-1]
if not os.path.exists(os.path.join(path, video_name)):
    os.mkdir(os.path.join(path, video_name))

# Set the desired downsampling rate (e.g., process every 5th frame)
downsampling_rate = cap.get(cv2.CAP_PROP_FPS)
frame_counter = 0

# Get the video's frames per second (fps)
fps = cap.get(cv2.CAP_PROP_FPS)
fps_multiplier = 1.0 / fps

while cap.isOpened():
    ret, curr_frame = cap.read()
    if not ret:
        break

    # Only process the frame if it is within the downsampling rate
    if frame_counter % downsampling_rate == 0:
        result = model(curr_frame)
        for res in result:
            boxes = res.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                if box.conf[0] > 0.5:
                    res_plotted = res[0].plot(conf=False)
                    current_frame_timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) * fps_multiplier / 1000
                    timedelta_obj = str(datetime.timedelta(seconds=(current_frame_timestamp)))
                    x = timedelta_obj.split(':')
                    hh, mm, ss = x[0], x[1], x[2]
                    filename = f"Timestamp - {hh} Hours, {mm} Minutes, {ss} Seconds.jpg"
                    filename = os.path.join(path, video_name, filename)
                    cv2.imwrite(filename, res_plotted)
                    print("Saved frame:", filename)

    # Increment the frame counter
    frame_counter += 1

cap.release()


0: 384x640 1 Sun Bears, 62.5ms
Speed: 0.0ms preprocess, 62.5ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Sun Bearss, 57.0ms
Speed: 11.2ms preprocess, 57.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00 Seconds.jpg
Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.033333 Seconds.jpg



0: 384x640 2 Sun Bearss, 52.1ms
Speed: 5.0ms preprocess, 52.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Sun Bearss, 58.0ms
Speed: 9.8ms preprocess, 58.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)


Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.066667 Seconds.jpg
Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.066667 Seconds.jpg
Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.100000 Seconds.jpg
Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.100000 Seconds.jpg



0: 384x640 1 Sun Bears, 58.0ms
Speed: 5.0ms preprocess, 58.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Sun Bearss, 59.0ms
Speed: 5.0ms preprocess, 59.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)



Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.166667 Seconds.jpg
Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.166667 Seconds.jpg


0: 384x640 1 Sun Bears, 58.0ms
Speed: 5.0ms preprocess, 58.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Sun Bears, 46.1ms
Speed: 5.6ms preprocess, 46.1ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)


Saved frame: C:\Users\HP\runs\detect\predict3\A Pair of Sun Bears (Helarctos malayanus) in Sumatra.mp4\Timestamp - 0 Hours, 00 Minutes, 00.200000 Seconds.jpg



0: 384x640 (no detections), 55.7ms
Speed: 5.0ms preprocess, 55.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.2ms
Speed: 8.9ms preprocess, 57.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.0ms
Speed: 18.5ms preprocess, 48.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.0ms
Speed: 6.0ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.0ms
Speed: 5.0ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.0ms
Speed: 5.0ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.0ms
Speed: 7.0ms preprocess, 59.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.1ms
Speed: 17.7ms preprocess, 49.1ms

In [8]:
from ultralytics import YOLO
import cv2
import os
import datetime

model = YOLO(r'C:\Users\HP\Desktop\bestd.pt')
source = r'C:\Users\HP\Desktop\Migrant boat capsize off Libyan coast caught on camera.mp4'
cap = cv2.VideoCapture(source)
path = r'C:\Users\HP\runs\detect\predict3'
frame_no = 0
video_name = source.split("\\")[-1]
if not os.path.exists(os.path.join(path, video_name)):
    os.mkdir(os.path.join(path, video_name))

# Set the desired downsampling rate (e.g., process every 5th frame)
downsampling_rate =1 # cap.get(cv2.CAP_PROP_FPS)
frame_counter = 0

# Get the video's frames per second (fps)
fps = cap.get(cv2.CAP_PROP_FPS)
fps_multiplier = 1.0 / 1.0 #fps

while cap.isOpened():
    ret, curr_frame = cap.read()
    if not ret:
        break

    # Only process the frame if it is within the downsampling rate
    if frame_counter % downsampling_rate == 0:
        result = model(curr_frame)
        for res in result:
            boxes = res.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                if box.conf[0]:
                    res_plotted = res[0].plot(conf=False)
                    current_frame_timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) * fps_multiplier / 1000
                    timedelta_obj = str(datetime.timedelta(seconds=(current_frame_timestamp)))
                    x = timedelta_obj.split(':')
                    hh, mm, ss = x[0], x[1], x[2]
                    filename = f"Timestamp - {hh} Hours, {mm} Minutes, {ss} Seconds.jpg"
                    filename = os.path.join(path, video_name, filename)
                    cv2.imwrite(filename, res_plotted)
                    print("Saved frame:", filename)

    # Increment the frame counter
    frame_counter += 1

cap.release()


0: 384x640 (no detections), 265.0ms
Speed: 4.0ms preprocess, 265.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 264.0ms
Speed: 4.0ms preprocess, 264.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 265.0ms
Speed: 4.0ms preprocess, 265.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 264.0ms
Speed: 5.0ms preprocess, 264.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 266.0ms
Speed: 4.0ms preprocess, 266.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 263.0ms
Speed: 4.0ms preprocess, 263.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 263.0ms
Speed: 3.0ms preprocess, 263.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 266.0ms
Speed: 4.0ms prepr

Saved frame: C:\Users\HP\runs\detect\predict3\Migrant boat capsize off Libyan coast caught on camera.mp4\Timestamp - 0 Hours, 00 Minutes, 00.640000 Seconds.jpg


0: 384x640 1 drowning_person, 264.0ms
Speed: 4.0ms preprocess, 264.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)



Saved frame: C:\Users\HP\runs\detect\predict3\Migrant boat capsize off Libyan coast caught on camera.mp4\Timestamp - 0 Hours, 00 Minutes, 00.680000 Seconds.jpg


0: 384x640 1 capsized_boat, 263.0ms
Speed: 5.0ms preprocess, 263.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)



...

0: 384x640 1 capsized_boat, 273.0ms
Speed: 4.0ms preprocess, 273.0ms inference, 11.0ms postprocess per image at shape (1, 3, 384, 640)



Saved frame: C:\Users\HP\runs\detect\predict3\Migrant boat capsize off Libyan coast caught on camera.mp4\Timestamp - 0 Hours, 00 Minutes, 09.840000 Seconds.jpg


0: 384x640 1 capsized_boat, 278.0ms
Speed: 4.0ms preprocess, 278.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)



Saved frame: C:\Users\HP\runs\detect\predict3\Migrant boat capsize off Libyan coast caught on camera.mp4\Timestamp - 0 Hours, 00 Minutes, 09.880000 Seconds.jpg


KeyboardInterrupt: 